In [1]:
import h5py
import numpy as np
import rasterio.features
import shapely.geometry
from osgeo import ogr
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from shapely.geometry import asPolygon
from shapely.geometry import asMultiPolygon


filename = "masks_prof.h5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    masks = list(f[a_group_key])

masks = np.array(masks)
print(masks.dtype)
masks = masks.astype(int)
print(masks.dtype)
#print(masks)

Keys: <KeysViewHDF5 ['masks']>
bool
int32


In [2]:
masks.shape

(246, 256, 14)

In [15]:
# baca tfw file dari image prediksi untuk dapat cell-size dan koordinat topleft
# tfw file berisi 6 baris: x-size, y-rotation, x-rotation, y-size, x-toplef, y-topleft
path_image = "./masks_shp_prof_multipoligon/tfw.tfw"
tfw = np.zeros((6))
i = 0
with open(path_image) as f:
    lines=f.readlines()
    for line in lines:
        tfw[i] = np.fromstring(line, dtype=float, sep=',')
        i = i+1
#print(tfw)

In [14]:
# COBAIN MENGATASI MULTIPOLYGON

count_shp=0
for i in range(masks.shape[2]):
    shapes = rasterio.features.shapes(masks[:,:,i])
    polygons = [shapely.geometry.Polygon(shape[0]["coordinates"][0]) for shape in shapes if shape[1] == 1]
    #print(len(polygons))
    if len(polygons)>1:
        for j in range(len(polygons)):
            polygons_sub = polygons[j]
            #print(polygons_sub)
    
            # mengubah ke UTM
            E,N = polygons_sub.exterior.coords.xy
            E = np.asarray(E)
            N = np.asarray(N)
            UTM_coords = np.zeros((len(E),2))
            for j in range(len(UTM_coords)):
                UTM_coords[j,0] = tfw[4] + E[j]*tfw[0]
                UTM_coords[j,1] = tfw[5] - N[j]*tfw[0]
            polygons_utm = asPolygon(UTM_coords)
            
            # menyimpan shapefile
            driver = ogr.GetDriverByName('Esri Shapefile')
            ds = driver.CreateDataSource('./masks_shp_prof_multipoligon/coord_utm_masks{:}.shp'.format(count_shp))
            layer = ds.CreateLayer('', None, ogr.wkbPolygon)
            # Add one attribute
            layer.CreateField(ogr.FieldDefn('id', ogr.OFTInteger))
            defn = layer.GetLayerDefn()

            ## If there are multiple geometries, put the "for" loop here

            # Create a new feature (attribute and geometry)
            feat = ogr.Feature(defn)
            feat.SetField('id', 123)

            # Make a geometry, from Shapely object
            geom = ogr.CreateGeometryFromWkb(polygons_utm.wkb) # nama polygonnya
            feat.SetGeometry(geom)

            layer.CreateFeature(feat)
            feat = geom = None  # destroy these

            # Save and close everything
            ds = layer = feat = geom = None            
            
            count_shp=count_shp+1
    
    else:
        polygons_sub = polygons[0]
        #print(polygons_sub[0].wkt)
        
        # mengubah ke UTM
        E,N = polygons_sub.exterior.coords.xy
        E = np.asarray(E)
        N = np.asarray(N)
        UTM_coords = np.zeros((len(E),2))
        for j in range(len(UTM_coords)):
            UTM_coords[j,0] = tfw[4] + E[j]*tfw[0]
            UTM_coords[j,1] = tfw[5] - N[j]*tfw[0]
        polygons_utm = asPolygon(UTM_coords)

        # menyimpan shapefile
        driver = ogr.GetDriverByName('Esri Shapefile')
        ds = driver.CreateDataSource('./masks_shp_prof_multipoligon/coord_utm_masks{:}.shp'.format(count_shp))
        layer = ds.CreateLayer('', None, ogr.wkbPolygon)
        # Add one attribute
        layer.CreateField(ogr.FieldDefn('id', ogr.OFTInteger))
        defn = layer.GetLayerDefn()

        ## If there are multiple geometries, put the "for" loop here

        # Create a new feature (attribute and geometry)
        feat = ogr.Feature(defn)
        feat.SetField('id', 123)

        # Make a geometry, from Shapely object
        geom = ogr.CreateGeometryFromWkb(polygons_utm.wkb) # nama polygonnya
        feat.SetGeometry(geom)

        layer.CreateFeature(feat)
        feat = geom = None  # destroy these

        # Save and close everything
        ds = layer = feat = geom = None            

        count_shp=count_shp+1

print("yeay finish")